In [1]:
#подгрузим требуемые библиотеки
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import csv
from tqdm import tqdm 

In [11]:
#сохраним User_agent и проверим код статуса ответа
HEADERS = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}
page = requests.get('https://www.lsgeotar.ru/pages/abc-pharma_tn.html', headers=HEADERS,verify=False, timeout=5)
page.status_code

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.lsgeotar.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


200

In [13]:
#Функция для получения HTML кода страницы
def get_url(url):
  headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}
  page = requests.get(url, headers=HEADERS, verify=False, timeout=20)
  return page.text

In [4]:
#Функция для получения ссылок на все страницы каталога, т.е. страница с препаратами на букву А,Б,В и т.д.
def get_all_pages(html):
  soup = BeautifulSoup(html, 'lxml')
  pages = soup.find('div', class_="abc_cyr").find_all('a')
  pages.extend(soup.find('div', class_="abc_latin").find_all('a'))
  pages.extend(soup.find('div', class_="abc_num").find_all('a'))
  pages_list = [pages[i].get('href') for i in range(len(pages))]
  return pages_list

In [17]:
#Функция для получения ссылок на все препараты страницы
def get_drug_list_url(html):
  soup = BeautifulSoup(html, 'lxml')
  pages = soup.find('div',class_='wrap-abc-items').find_all('div', class_='clearfix wrap-abc-pharma')
  medication_urls = [pages[i].find('a', class_='title-tn-link').get('href') for i in range(len(pages))]
  return medication_urls

In [95]:
#Ну и собственно парсим нужные нам поля из карточки
def get_medication_info(html):
  soup = BeautifulSoup(html, 'lxml')
  try:
    name = soup.find('div', class_="name-tn-mnn-text").find('span', class_="value").text
  except:
    name = ''
  try:
    substance = soup.find('article',class_='all-article-preparat ls-preparat-tn col-md-9 col-sm-12 col-xs-12').find('span', class_='value').text
  except:
    substance = ''
  try:
    form = soup.find('section', class_="ls-dosage-form general-section").find('span', class_="value").text
  except:
    form = ''
  try:
    description = soup.find('section',class_='ls-pokazaniya general-section').find('span', class_='value').text
  except:
    description = ''
  try:
    contraindications = soup.find('div',class_='container-content-preparat').find('section', class_='ls-protivopokazaniya general-section').find('p').text
  except:
    contraindications = ''

  return name, substance, form, description, contraindications

In [83]:
#def write_to_csv(data):
 # with open('drugs_data.csv', 'a') as f:
  #  writer = csv.writer(f)
#
 #   writer.writerow( (data['name'],
  #                    data['substance'],
   #                   data['description'],
    #                  data['contraindication'],
     #                 data['url']))

Сохранял датасет сначала через контекстный менеджер, но в формат данных при таком сохранении у меня на компьютере оказался плохо читаемым. В итоге просто объединил все данные в датасет и сохранил при помощи to_csv()

In [ ]:
# Запускаем получившися скипт и собственно парсим требуемы данные с сайта, пока просто по спискам
if __name__=="__main__":
  medication_urls = []
  names = []
  substances = []
  forms = []
  descriptions = []
  contraindication_1 = []
  for i in get_all_pages(get_url('https://www.lsgeotar.ru/pages/abc-pharma_tn.html')):
    for j in tqdm(get_drug_list_url(get_url(i))):
      medication_urls.append(j)
      info = get_medication_info(get_url(j))
      names.append(info[0])
      substances.append(info[1])
      forms.append(info[2])
      descriptions.append(info[3])
      contraindication_1.append(info[4])

In [84]:
# Объединим все полученные переменные в один словарь
data = {'name': names,
        'substance': substances,
        'form': forms,
        'description':descriptions,
        'contraindication':contraindication_1,
        'url':medication_urls}

In [85]:
#write_to_csv(data)

In [86]:
df = pd.DataFrame(data)

In [90]:
df.to_csv('drug_df.csv')

In [18]:
data = pd.read_excel('/content/drive/MyDrive/Doc24/Drugs/drug_df.xlsx')

In [26]:
#Собственно результат
data.drop(['Unnamed: 0','url'], axis=1).head()

,name,substance,description,contraindication
0,Б-190,Индолилметиламиноэтанол,Б-190 применяют в качестве профилактического р...,Б-190 противопоказан при артериальной гипертон...
1,Бавенсио,Авелумаб,Монотерапия у взрослых ранее леченных пациенто...,- Гиперчувствительность к препарату БАВЕНСИО и...
2,Багодилол,Карведилол,- Артериальная гипертензия (монотерапия и в ко...,Повышенная чувствительность к карведилолу или ...
3,Багомет Плюс,Глибенкламид + Метформин,Сахарный диабет 2 типа у взрослых: в качест...,
4,Багомет,Метформин,"Сахарный диабет 2 типа, особенно в сочетании с...",- Повышенная чувствительность к метформину или...
